In [2]:
!pip install catboost -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.2 MB/s eta 0:00:00


In [6]:
# ===== Win LB: KFold Target Encoding + XGBoost (+CatBoost if available) =====
import numpy as np
import pandas as pd
import xgboost as xgb
import warnings
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore")

# Try CatBoost; continue without if missing (prints same message as before)
CAT_OK = True
try:
    from catboost import CatBoostRegressor, Pool
except Exception:
    CAT_OK = False
    print("ℹ️ CatBoost not found (ok). To enable: pip install catboost")

# ---------------- Config ----------------
SEED, FOLDS = 42, 5
TARGET = "efficiency"

# ---------------- Load ----------------
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
sub   = pd.read_csv("sample_submission.csv")

feat   = [c for c in train.columns if c != TARGET]
X      = train[feat].copy()
y      = train[TARGET].copy()
X_test = test[feat].copy()

num_cols = X.select_dtypes(include="number").columns.tolist()
cat_cols = X.select_dtypes(include="object").columns.tolist()

# ---------------- Clean ----------------
# Numeric impute (fit on train, apply to test)
num_imp = SimpleImputer(strategy="median")
X[num_cols]      = num_imp.fit_transform(X[num_cols])
X_test[num_cols] = num_imp.transform(X_test[num_cols])

# Categorical NA to "NA"
for c in cat_cols:
    X[c]      = X[c].astype(str).fillna("NA")
    X_test[c] = X_test[c].astype(str).fillna("NA")

# Rare merge (helps generalization) — same logic
def merge_rare(s: pd.Series, min_count=20):
    vc = s.value_counts()
    return s.where(s.map(vc) >= min_count, "RARE")

for c in cat_cols:
    X[c] = merge_rare(X[c])
    seen = set(X[c].unique())
    X_test.loc[~X_test[c].isin(seen), c] = "RARE"

# Target bounds (used for clipping at the end)
y_min, y_max = float(y.min()), float(y.max())

# ---------------- Optional log1p target ----------------
USE_LOG = (y > 0).all()
if USE_LOG:
    y_train_fit = np.log1p(y)
    print("🔁 Using log1p target.")
else:
    y_train_fit = y.values

# ---------------- KFold Target Encoding ----------------
# Per-fold mean encoding with smoothing; applied to train folds and test using full-train means.
def kfold_target_encode(train_df, test_df, y_vec, cats, n_splits=FOLDS, seed=SEED, smoothing=20.0):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof_te  = pd.DataFrame(index=train_df.index)
    test_te = pd.DataFrame(index=test_df.index)
    global_mean = y_vec.mean()

    for c in cats:
        oof_col  = np.zeros(len(train_df))
        test_col = np.zeros(len(test_df))

        for tr_idx, vl_idx in kf.split(train_df):
            tr, vl = train_df.iloc[tr_idx], train_df.iloc[vl_idx]
            y_tr   = y_vec.iloc[tr_idx] if isinstance(y_vec, pd.Series) else y_vec[tr_idx]

            counts = tr[c].value_counts()
            means  = tr.join(pd.Series(y_tr, index=tr.index).rename("__y")).groupby(c)["__y"].mean()

            # smoothing: (count*mean + s*global) / (count + s)
            denom   = counts.reindex(means.index).fillna(0.0)
            sm_mean = (means * denom + smoothing * global_mean) / (denom + smoothing)

            oof_col[vl_idx] = vl[c].map(sm_mean).fillna(global_mean).values

        # test encoding: fit on full train
        counts_full = train_df[c].value_counts()
        means_full  = train_df.join(pd.Series(y_vec, index=train_df.index).rename("__y")).groupby(c)["__y"].mean()
        denom_full  = counts_full.reindex(means_full.index).fillna(0.0)
        sm_full     = (means_full * denom_full + smoothing * global_mean) / (denom_full + smoothing)

        test_col = test_df[c].map(sm_full).fillna(global_mean).values

        oof_te[f"TE__{c}"]  = oof_col
        test_te[f"TE__{c}"] = test_col

    return oof_te, test_te

# Build TE features (use the same target form we train on)
y_for_te = pd.Series(y_train_fit, index=X.index) if USE_LOG else pd.Series(y, index=X.index)
TE_tr, TE_te = kfold_target_encode(X, X_test, y_for_te, cat_cols)

# Final matrices for XGB: [numerics + TE] (no OHE, same as original)
X_xgb     = np.hstack([X[num_cols].values,      TE_tr.values])
Xtest_xgb = np.hstack([X_test[num_cols].values, TE_te.values])

# ---------------- XGBoost (5-fold CV) ----------------
params = dict(
    objective="reg:squarederror",
    learning_rate=0.03,
    max_depth=7,
    min_child_weight=8,
    subsample=0.85,
    colsample_bytree=0.85,
    reg_lambda=3.0,
    reg_alpha=0.1
)

kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
oof_xgb    = np.zeros(len(X))
pred_xgb   = np.zeros(len(X_test))
best_rounds = []

for fold, (tr_idx, vl_idx) in enumerate(kf.split(X_xgb), 1):
    dtr = xgb.DMatrix(X_xgb[tr_idx], label=(y_train_fit[tr_idx] if USE_LOG else y.iloc[tr_idx]))
    dvl = xgb.DMatrix(X_xgb[vl_idx],  label=(y_train_fit[vl_idx] if USE_LOG else y.iloc[vl_idx]))

    m = xgb.train(params, dtr, num_boost_round=8000,
                  evals=[(dvl, "val")], early_stopping_rounds=400, verbose_eval=False)

    best_rounds.append(m.best_iteration + 1)
    oof_xgb[vl_idx] = m.predict(dvl)

    # Fold-wise model on full data for test contribution (same logic)
    pred_xgb += xgb.train(params, xgb.DMatrix(X_xgb, label=(y_train_fit if USE_LOG else y)),
                          num_boost_round=m.best_iteration + 1, verbose_eval=False) \
                          .predict(xgb.DMatrix(Xtest_xgb)) / FOLDS

# Undo log if used (same function/behavior)
def inv_t(z):
    return np.expm1(z) if USE_LOG else z

oof_xgb_real  = inv_t(oof_xgb)
pred_xgb_real = inv_t(pred_xgb)

rmse_xgb = mean_squared_error(y, oof_xgb_real) ** 0.5
print(f"✅ XGB OOF RMSE: {rmse_xgb:.6f} | mean trees: {np.mean(best_rounds):.0f}")

# ---------------- CatBoost (optional, if installed) ----------------
use_cat = CAT_OK and (len(cat_cols) > 0)
if use_cat:
    oof_cat  = np.zeros(len(X))
    pred_cat = np.zeros(len(X_test))

    params_cat = dict(
        loss_function="RMSE",
        learning_rate=0.03,
        depth=8,
        l2_leaf_reg=6.0,
        iterations=20000,
        od_type="Iter",
        od_wait=400,
        random_seed=SEED,
        verbose=False
    )

    for tr_idx, vl_idx in kf.split(X):
        tr_pool = Pool(
            X.iloc[tr_idx],
            (np.log1p(y.iloc[tr_idx]) if USE_LOG else y.iloc[tr_idx]),
            cat_features=[X.columns.get_loc(c) for c in cat_cols]
        )
        vl_pool = Pool(
            X.iloc[vl_idx],
            (np.log1p(y.iloc[vl_idx]) if USE_LOG else y.iloc[vl_idx]),
            cat_features=[X.columns.get_loc(c) for c in cat_cols]
        )

        m = CatBoostRegressor(**params_cat)
        m.fit(tr_pool, eval_set=vl_pool, use_best_model=True)

        oof_cat[vl_idx]  = m.predict(vl_pool)
        pred_cat        += m.predict(Pool(X_test, cat_features=[X.columns.get_loc(c) for c in cat_cols])) / FOLDS

    oof_cat_real  = inv_t(oof_cat)
    pred_cat_real = inv_t(pred_cat)
    rmse_cat = mean_squared_error(y, oof_cat_real) ** 0.5
    print(f"✅ CAT OOF RMSE: {rmse_cat:.6f}")

# ---------------- OOF-based blending ----------------
if use_cat:
    alphas = np.linspace(0, 1, 51)  # weight on XGB
    best_rmse, best_a = 1e9, None
    for a in alphas:
        blend_oof = a * oof_xgb_real + (1 - a) * oof_cat_real
        r = mean_squared_error(y, blend_oof) ** 0.5
        if r < best_rmse:
            best_rmse, best_a = r, a
    print(f"🥇 Blend OOF RMSE: {best_rmse:.6f} (alpha={best_a:.2f} → XGB weight)")
    final_pred = best_a * pred_xgb_real + (1 - best_a) * pred_cat_real
else:
    final_pred = pred_xgb_real

# ---------------- Clip + Submit ----------------
# If your target is truly [0,1], set these explicitly to (0.0, 1.0)
final_pred = np.clip(final_pred, y_min, y_max)

if "efficiency" not in sub.columns:
    raise ValueError("sample_submission.csv needs 'efficiency' column.")
if len(sub) != len(final_pred):
    raise ValueError("Row mismatch between sample_submission and predictions.")

sub["efficiency"] = final_pred
sub.to_csv("submission.csv", index=False)


🔁 Using log1p target.
✅ XGB OOF RMSE: 0.881561 | mean trees: 301
✅ CAT OOF RMSE: 0.787303
🥇 Blend OOF RMSE: 0.787303 (alpha=0.00 → XGB weight)
📄 Saved submission.csv
   id  efficiency
0   1    8.304069
1   4   10.549170
2   5   12.320598
3   9    5.000000
4  11   14.681001
